In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.7 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import faiss
import numpy as np

def create_faiss_index(embeddings):
    """
    코사인 유사도용 FAISS 인덱스를 생성하는 함수

    Args:
        embeddings (np.ndarray): 임베딩 벡터 (2D 배열)

    Returns:
        index (faiss.IndexFlatIP): FAISS 인덱스
    """
    # 벡터 정규화 (코사인 유사도 계산을 위한 과정)
    faiss.normalize_L2(embeddings)

    # 내적(Inner Product) 기반 인덱스 생성 (코사인 유사도 계산 가능)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)

    # 인덱스에 임베딩 추가
    index.add(embeddings)

    return index

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# BERT 모델 저장
model.save_pretrained("./bert_model")
tokenizer.save_pretrained("./bert_model")

def get_bert_embedding(text):
    """
    BERT 임베딩 생성 함수

    Args:
        text (str): 임베딩할 텍스트

    Returns:
        embedding (np.ndarray): 임베딩 벡터 (1D 배열)
    """
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)

    # [CLS] 토큰의 임베딩 사용 (문장 전체 의미 표현)
    embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()

    return embedding.astype('float32')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# GPT-neo 로드
gpt_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
gpt_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

# GPT-Neo 모델 저장
gpt_model.save_pretrained("./gpt_neo_model")
gpt_tokenizer.save_pretrained("./gpt_neo_model")

def generate_gpt_response(context, question):
    """
    GPT-neo로 Q&A 응답 생성

    Args:
        context (str): Q&A 검색 결과로 얻은 문맥
        question (str): 사용자 질문

    Returns:
        response (str): GPT-neo 응답 결과
    """
    prompt = ("다음은 Q&A 자동응답 시스템의 문맥과 질문입니다. 문맥을 바탕으로 질문에 답하세요.\n\n"f"문맥: {context}\n질문: {question}\n답변: ")
    inputs = gpt_tokenizer(prompt, return_tensors='pt', max_length=1024, truncation=True)
    outputs = gpt_model.generate(inputs.input_ids, max_length=1024, num_return_sequences=1, early_stopping=True, pad_token_id=gpt_tokenizer.eos_token_id)

    response = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True).split("답변:")[-1].strip()
    return response

In [ ]:
def qa_pipeline(question, index, embeddings, qa_texts, k=5):
    """
    Q&A 자동응답 시스템 파이프라인

    Args:
        question (str): 사용자 질문
        index (faiss.IndexFlatIP): FAISS 인덱스
        embeddings (np.ndarray): 기존 Q&A 임베딩 벡터
        qa_texts (list): 기존 Q&A 텍스트 리스트
        k (int): 상위 몇 개의 유사 Q&A를 가져올지

    Returns:
        gpt_response (str): GPT-neo로 생성한 응답
    """

    # 1. 질문 임베딩 생성
    query_vector = get_bert_embedding(question)

    # 2. 벡터 정규화 (코사인 유사도 계산)
    faiss.normalize_L2(query_vector)
    print("[query_vector]")
    print(query_vector)
    # 3. FAISS 검색 (상위 k개 결과 반환)
    distances, indices = index.search(query_vector, k)
    print("[distances]")
    print(distances)
    print("[indices]")
    print(indices)
    # 4. 가장 유사한 Q&A 항목들 추출
    similar_texts = [qa_texts[i] for i in indices[0]]
    print("[similar_texts]")
    print(similar_texts)
    # 5. GPT-neo에게 문맥과 질문 제공, 응답 생성
    context = " ".join(similar_texts)
    gpt_response = generate_gpt_response(context, question)

    return gpt_response

In [ ]:
# 1. 기존 Q&A 데이터
qa_texts = [
    "AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다.",
    "FAISS는 Facebook AI에서 만든 고차원 벡터 검색 라이브러리입니다.",
    "BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다.",
    "GPT-neo는 언어 생성을 위한 트랜스포머 모델로, 대규모 데이터를 바탕으로 훈련되었습니다.",
    "코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다."
]

# 2. Q&A 임베딩 생성
embeddings = np.vstack([get_bert_embedding(text) for text in qa_texts])
print("[embeddings]")
print(embeddings)
# 3. FAISS 인덱스 생성
index = create_faiss_index(embeddings)
print("[index]")
print(index)
# 4. 질문 생성 및 응답 확인
question = "인공지능의 줄임말로, 기계가 사람처럼"
response = qa_pipeline(question, index, embeddings, qa_texts)
print("GPT-neo 응답:", response)

[embeddings]
[[-0.50681555 -0.2028872  -0.33815995 ...  0.01465861  0.81922936
  -0.09951616]
 [-0.43236676 -0.40407437 -0.19400476 ... -0.10309409  0.814261
  -0.03031765]
 [-0.5119181  -0.25768512 -0.33776063 ...  0.03305429  0.84757555
  -0.15074971]
 [-0.50990474 -0.33507633 -0.01816359 ... -0.11603286  0.90274453
  -0.06597818]
 [-0.47904012 -0.26681828 -0.43792173 ...  0.07808858  0.86179936
   0.00239504]]
[index]
<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x7a1ef623f5a0> >
[query_vector]
[[-2.83597279e-02 -9.33401659e-03 -2.93455683e-02 -2.23542117e-02
  -5.48630618e-02 -6.61463523e-03  6.62154257e-02  3.86597663e-02
   6.94077788e-03 -2.45053656e-02  8.23835551e-04 -2.14492599e-03
  -1.30842915e-02  1.74964208e-03  1.00416327e-02  1.06031699e-02
  -7.17888540e-03  1.72326677e-02  2.54053827e-02  1.66453831e-02
  -4.33237106e-02  1.72337436e-03 -7.24914437e-03 -4.71551670e-03
   8.59881379e-03 -3.20536904e-02 -1.29767526e-02  2.11

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


GPT-neo 응답: 기계가 사람처럼

질문: 인공지능의 줄임말로, 기계가 사람처럼
�


In [ ]:
from sentence_transformers import SentenceTransformer

# Sentence-BERT 모델 로드 (다양한 모델이 존재, 예시로 'paraphrase-MiniLM-L6-v2' 모델 사용)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 문장 리스트
sentences = [
    "AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다.",
    "FAISS는 Facebook AI에서 만든 고차원 벡터 검색 라이브러리입니다.",
    "BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다.",
    "GPT-neo는 언어 생성을 위한 트랜스포머 모델로, 대규모 데이터를 바탕으로 훈련되었습니다.",
    "코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다."
]

# 문장 임베딩 생성
embeddings = model.encode(sentences)

# 임베딩 결과 출력 (각 문장에 대한 고차원 벡터)
for sentence, embedding in zip(sentences, embeddings):
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embedding[:5]}...")  # 첫 5개 값만 출력
    print()


Sentence: AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다.
Embedding: [ 0.45696622  0.33455956 -0.02110084 -0.37054583 -0.01043585]...

Sentence: FAISS는 Facebook AI에서 만든 고차원 벡터 검색 라이브러리입니다.
Embedding: [ 0.30969518 -0.1030289  -0.02162404 -0.39943364  0.17117104]...

Sentence: BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다.
Embedding: [ 0.00250799  0.1817315   0.12246078 -0.48240036 -0.05990716]...

Sentence: GPT-neo는 언어 생성을 위한 트랜스포머 모델로, 대규모 데이터를 바탕으로 훈련되었습니다.
Embedding: [-0.00259389  0.06711136 -0.3267618  -0.50269693 -0.25174183]...

Sentence: 코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다.
Embedding: [ 0.5120305   0.28432947  0.16044842 -0.20529318 -0.15759869]...



In [ ]:
import faiss
import numpy as np

# FAISS에서 사용할 임베딩 벡터 배열 (벡터를 L2 정규화)
embeddings = np.array(embeddings).astype(np.float32)

# 벡터 정규화
faiss.normalize_L2(embeddings)

# FAISS 인덱스 생성 (내적을 사용할 때는 IndexFlatIP를 사용)
index = faiss.IndexFlatIP(embeddings.shape[1])  # 임베딩 벡터의 차원

# FAISS 인덱스에 임베딩 추가
index.add(embeddings)

# 쿼리 문장
query_sentence = "FAISS로 코사인 유사도를 계산한다는데 이게 무슨 말이야?"

# 쿼리 임베딩 생성
query_embedding = model.encode([query_sentence]).astype(np.float32)

# 쿼리 벡터 정규화
faiss.normalize_L2(query_embedding)

# 유사도 검색 (가장 유사한 3개 문장 검색)
distances, indices = index.search(query_embedding, 3)

# 결과 출력
print("Query:", query_sentence)
for i in range(3):
    print(f"Top {i+1}: {sentences[indices[0][i]]} (Distance: {distances[0][i]:.4f})")

similar_texts = [sentences[i] for i in indices[0]]
print(similar_texts)
context = " ".join(similar_texts)
gpt_response = generate_gpt_response(context, query_sentence)
print(gpt_response)

Query: FAISS로 코사인 유사도를 계산한다는데 이게 무슨 말이야?
Top 1: BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다. (Distance: 0.8563)
Top 2: 코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다. (Distance: 0.8549)
Top 3: AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다. (Distance: 0.8473)
['BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다.', '코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다.', 'AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다.']
